# Preliminary Sentence Bert Analysis

Arthor: Crystal

In [5]:
#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

import re

import nltk
#nltk.download() #input: punkt

from nltk import tokenize

In [21]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [11]:
# functions to clean text
def is_empty(texts):
    if (texts =='\n') or (texts=='\r'):
        return True
    else:
        return False

#clen wiki text
def clean_wiki_text(texts):
    if is_empty(texts):
        return None
    else:
        texts = re.sub(r'\[[0-9]*\]', ' ', texts) #removes numbers
        texts = re.sub(r'\s+',' ',texts) #remove long blank space
        return(texts)

## AI article

In [18]:
with open("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/ai_wiki_text.txt") as f:
    ai_text = f.read()

### Sentence Tokenization

In [19]:
ai_corpus = tokenize.sent_tokenize(ai_text)

### Embedding

In [22]:
ai_embeddings = embedder.encode(ai_corpus, show_progress_bar=False)

## AI related articles

In [38]:
articles = pd.read_csv("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/wiki_ai_related_articles.csv") #import csv

texts = articles["article"] #get the text

text_clean_ls = []
for text in texts:
    text_clean = clean_wiki_text(text)
    text_clean_ls.append(text_clean)

### Example 1.

We feed in ML Wiki article as a query.
For each sentence in the ML Wiki article, we identify the top 5 most similar/closest sentences from AI Wiki article based on cosine similarity

In [60]:
# Example
ml_p = text_clean_ls[9]
queries = tokenize.sent_tokenize(ml_p)
print(queries[0:10])
print(len(queries))

['Machine learning (ML) is the study of computer algorithms that improve automatically through experience and by the use of data.', 'It is seen as a part of artificial intelligence.', 'Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.', 'Machine learning algorithms are used in a wide variety of applications, such as in medicine, email filtering, speech recognition, and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.', 'A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers; but not all machine learning is statistical learning.', 'The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.', 'Data mining is a related field of study, focusing on exploratory dat

In [58]:
# Find the closest 5 sentences of the AI corpus for each query sentence (ML) based on cosine similarity
top_k = min(5, len(ai_corpus))
for query in queries:
    #query_embedding = embedder.encode(query, convert_to_tensor=True)
    query_embedding = embedder.encode(query, show_progress_bar=False)
    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(ai_corpus[idx], "(Score: {:.4f})".format(score))





Query: Machine learning (ML) is the study of computer algorithms that improve automatically through experience and by the use of data.

Top 5 most similar sentences in corpus:
Machine learning (ML), a fundamental concept of AI research since the field's inception,[d] is the study of computer algorithms that improve automatically through experience. (Score: 0.8646)
A more elaborate definition characterizes AI as "a system's ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation." (Score: 0.7156)
Many tools are used in AI, including versions of search and mathematical optimization, artificial neural networks, and methods based on statistics, probability and economics. (Score: 0.7017)
Approaches include statistical methods, computational intelligence, and traditional symbolic AI. (Score: 0.6751)
AI often revolves around the use of algorithms. (Score: 0.6645)




Query: It is seen

## Example 2. (Abstract)

In [6]:
abstracts = pd.read_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/smaller-final-dataset.pkl")

In [13]:
abstract_text = abstracts["ABSTRACT"][0]#get the text
abstract_text

"The multiprotein complex y-secretase proteolytically cleaves the intramembrane region of amyloid precursorprotein (APP), which in turn forms the plaques found in Alzheimer's disease (AD) patients. The catalyticcomponent of Y-secretase is the intramembrane aspartyl protease (IAP) called presenilin. Mutations inpresenilin are directly linked to familial early-onset AD. Another known member of the IAP family is signalpeptide peptidase (SPP), which functions to further proteolyze remnant signal peptides after they have beencleaved by signal peptidase. Knowledge of the biochemistry and function of individual SPPs are onlybeginning to be elucidated, and homologues are found in all kingdoms of life. Presenilin and SPP exhibitsignificant sequence similarity, strongly suggesting they share structural and catalytic features. Thus, amolecular understanding of the more tractable SPP will likely impact drug design for presenilin and y-secretase. The goal of this proposal is to express, characteriz

In [16]:
queries = tokenize.sent_tokenize(abstract_text)
print(queries[0:10])
print(len(queries))

["The multiprotein complex y-secretase proteolytically cleaves the intramembrane region of amyloid precursorprotein (APP), which in turn forms the plaques found in Alzheimer's disease (AD) patients.", 'The catalyticcomponent of Y-secretase is the intramembrane aspartyl protease (IAP) called presenilin.', 'Mutations inpresenilin are directly linked to familial early-onset AD.', 'Another known member of the IAP family is signalpeptide peptidase (SPP), which functions to further proteolyze remnant signal peptides after they have beencleaved by signal peptidase.', 'Knowledge of the biochemistry and function of individual SPPs are onlybeginning to be elucidated, and homologues are found in all kingdoms of life.', 'Presenilin and SPP exhibitsignificant sequence similarity, strongly suggesting they share structural and catalytic features.', 'Thus, amolecular understanding of the more tractable SPP will likely impact drug design for presenilin and y-secretase.', 'The goal of this proposal is t

In [23]:
# Find the closest 5 sentences of the AI corpus for each query sentence (ML) based on cosine similarity
top_k = min(5, len(ai_corpus))
for query in queries:
    #query_embedding = embedder.encode(query, convert_to_tensor=True)
    query_embedding = embedder.encode(query, show_progress_bar=False)
    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(ai_corpus[idx], "(Score: {:.4f})".format(score))





Query: The multiprotein complex y-secretase proteolytically cleaves the intramembrane region of amyloid precursorprotein (APP), which in turn forms the plaques found in Alzheimer's disease (AD) patients.

Top 5 most similar sentences in corpus:
DeepMind's AlphaFold 2 (2020) demonstrated the ability to determine, in hours rather than months, the 3D structure of a protein. (Score: 0.3053)
Regulation of AI through mechanisms such as review boards can also be seen as social means to approach the AI control problem. (Score: 0.2793)
The regulation of artificial intelligence is the development of public sector policies and laws for promoting and regulating artificial intelligence (AI);   it is therefore related to the broader regulation of algorithms. (Score: 0.2571)
The hard problem is explaining how the brain creates it, why it exists, and how it is different from knowledge and other aspects of the brain. (Score: 0.2551)
Knowledge representation  and knowledge engineering  are central t